## Plot maps with solution (dam porftolios) for different power generation targets
#### T. Janus
#### Phan Rang 06.02.2024

In [ ]:
import pathlib
import matplotlib
import warnings
from math import pi
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from shapely.geometry import Point
import matplotlib.gridspec as gridspec

### Load data

In [ ]:
# 1. Load elevation data and basin data from ifc database
ifcdb_df = gpd.read_file(pathlib.Path("bin/gis_layers/ifc_database/all_dams_replaced_refactored.shp"))\
    .loc[:, ['IFC_ID', 'DAM_NAME', 'Basin', 'Sub-Basin', 'Sub-Basi_1', 'RIV_ORD', 'FSL (m)']]\
    .set_index('IFC_ID')
# Set missing FSL values from Pywr model data
ifcdb_df.at[8, 'FSL (m)'] = 313 #400
ifcdb_df.at[10, 'FSL (m)'] = 323.0 #428
ifcdb_df.at[120, 'FSL (m)'] = 178

# 2. Load outputs file with MOO results
outputs_file = pathlib.Path("intermediate/dams_for_plotting_moo.csv")
output_df = pd.read_csv(outputs_file, index_col=None)
# output_df has duplicated ifc ids. Therefore, assign ifc_id of Mali dam to 236
output_df.loc[output_df['name'] == "Mali", "ifc_id"] = 236
output_df.set_index("ifc_id", inplace=True)
# Define bin edges for land loss
bins = [0, 3.1, 20, 40, 60, 100, 400, 800, 5000]
# Define labels for the bins
#labels = ['0-400 km2', '400-800 km2', '800-1200 km2', '1200-1600 km2']
output_df['GHG intensity cat [gCO2,eq/kWh]'] = pd.cut(
    output_df['GHG intensity [gCO2,eq/kWh]'], bins=bins)

# 3. Load file with selected dams per optimization scenario
with open(pathlib.Path('intermediate/optim_scenarios.json'), 'r') as file:
    optim_scenarios = json.load(file)
    
df_combined = ifcdb_df

In [ ]:
output_df_riv_ord = output_df.reset_index().merge(
    ifcdb_df.reset_index()[['IFC_ID', 'RIV_ORD']], left_on="ifc_id", right_on="IFC_ID",
    how="left").set_index("ifc_id")

## Helper functions

In [ ]:
class SeabornFig2Grid():

    def __init__(self, seaborngrid, fig,  subplot_spec):
        self.fig = fig
        self.sg = seaborngrid
        self.subplot = subplot_spec
        if isinstance(self.sg, sns.axisgrid.FacetGrid) or \
            isinstance(self.sg, sns.axisgrid.PairGrid):
            self._movegrid()
        elif isinstance(self.sg, sns.axisgrid.JointGrid):
            self._movejointgrid()
        self._finalize()

    def _movegrid(self):
        """ Move PairGrid or Facetgrid """
        self._resize()
        n = self.sg.axes.shape[0]
        m = self.sg.axes.shape[1]
        self.subgrid = gridspec.GridSpecFromSubplotSpec(n,m, subplot_spec=self.subplot)
        for i in range(n):
            for j in range(m):
                self._moveaxes(self.sg.axes[i,j], self.subgrid[i,j])

    def _movejointgrid(self):
        """ Move Jointgrid """
        h= self.sg.ax_joint.get_position().height
        h2= self.sg.ax_marg_x.get_position().height
        r = int(np.round(h/h2))
        self._resize()
        self.subgrid = gridspec.GridSpecFromSubplotSpec(r+1,r+1, subplot_spec=self.subplot)

        self._moveaxes(self.sg.ax_joint, self.subgrid[1:, :-1])
        self._moveaxes(self.sg.ax_marg_x, self.subgrid[0, :-1])
        self._moveaxes(self.sg.ax_marg_y, self.subgrid[1:, -1])

    def _moveaxes(self, ax, gs):
        #https://stackoverflow.com/a/46906599/4124317
        ax.remove()
        ax.figure=self.fig
        self.fig.axes.append(ax)
        self.fig.add_axes(ax)
        ax._subplotspec = gs
        ax.set_position(gs.get_position(self.fig))
        ax.set_subplotspec(gs)

    def _finalize(self):
        plt.close(self.sg.fig)
        self.fig.canvas.mpl_connect("resize_event", self._resize)
        self.fig.canvas.draw()

    def _resize(self, evt=None):
        self.sg.fig.set_size_inches(self.fig.get_size_inches())

In [ ]:
def plot_mya_reservoirs(
        data: pd.DataFrame,
        ax: matplotlib.axes.Axes,
        column_name: str,
        lon_field: str = "coordinates_1",
        lat_field: str = "coordinates_0",
        rivers_shp: str | None = r'bin/gis_layers/mya_rivers.shp',
        outline_shp: str | None = r'bin/gis_layers/myanmar_outline/Myanmar_outline.shp',
        res_label_field: str | None = None,
        title: str | None = None,
        title_font_size: int = 14,
        marker_size: int | str = 40,
        marker_size_multiplier: float = 1.0,
        cmap = 'tab10',
        plot_legends = (False, False, False),
        offset: float = 0.0,
        legend_y_coords = (1.0, 0.2, -0.05),
        **kwargs):
    """Plots the maps with dams"""
    crs={'init':'epsg:4326'}
    from matplotlib.lines import Line2D
    geometry=[Point(xy) for xy in zip(data[lon_field], data[lat_field])]
    data_gdf=gpd.GeoDataFrame(data,crs=crs, geometry=geometry)
    if rivers_shp is not None:
        rivers = gpd.read_file(rivers_shp, crs=crs)
        rivers.plot(ax=ax, edgecolor = 'k', linewidth=0.3, alpha = 0.5)
    if outline_shp is not None:
        outline = gpd.read_file(outline_shp)
        outline.plot(ax=ax, facecolor='Grey', edgecolor='k',alpha=0.05,linewidth=1,cmap="cividis")
        outline.plot(ax=ax, facecolor='none', edgecolor='k',alpha=1,linewidth=0.2,cmap="cividis")
    # Divide dams into storage and RoR
    data_gdf_ror = data_gdf[data_gdf['hp_type_reem'] == 'ror']
    data_gdf_sto = data_gdf[data_gdf['hp_type_reem'] == 'sto']
    
    data_gdf_sto_hp = data_gdf[(data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'hydroelectric')]
    data_gdf_sto_mp = data_gdf[(data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'multipurpose')]
    
    if isinstance(marker_size, str):
        marker_size_ror = np.sqrt(data_gdf_ror[marker_size].astype(float))
        marker_size_sto = np.sqrt(data_gdf_sto[marker_size].astype(float))
        marker_size_sto_hp = np.sqrt(data_gdf_sto_hp[marker_size].astype(float))
        marker_size_sto_mp = np.sqrt(data_gdf_sto_mp[marker_size].astype(float))
        
    p1 = data_gdf_ror.plot(
        kind='geo', ax=ax, column=column_name, cmap=cmap, 
        markersize=marker_size_ror*marker_size_multiplier,
        marker = "^", k=10,
        linewidth=0.5, edgecolor='k', alpha=0.45, categorical = True, 
        legend_kwds={
            'loc':'upper right', 
            'bbox_to_anchor': (1.17+offset, legend_y_coords[0]),
            'markerscale':0.5, 
            'title_fontsize':'medium', 
            'frameon':False,
            'fontsize':"small"},
        legend=plot_legends[0],
        **kwargs)
    p2 = data_gdf_sto_hp.plot(
        kind = 'geo', ax=ax, column=column_name, cmap=cmap, 
        markersize=marker_size_sto_hp*marker_size_multiplier, 
        marker = "o", k = 10,
        linewidth=0.5, edgecolor='k', alpha=0.45, categorical = True,        
        **kwargs)
    p3 = data_gdf_sto_mp.plot(
        kind = 'geo', ax=ax, column=column_name, cmap=cmap, aspect=1,
        markersize=marker_size_sto_mp*marker_size_multiplier, 
        marker = "s", k = 10,
        linewidth=0.5, edgecolor='k', alpha=0.45, categorical = True, **kwargs)
    
    if plot_legends[0]:
        leg1 = p1.get_legend()
        leg1._legend_box.align = "left"
        custom_labels = ["0.0-3.1", "3.1-20", "20-40", "40-60", "60-100", "100-400", "400-800", "800-5000"]
        if leg1:
            leg1.set_title("GHG Emission\nIntensity\n[gCO$_{2,eq}$/kWh]")
        if leg1 and custom_labels:
            new_legtxt = custom_labels
            for ix,eb in enumerate(leg1.get_texts()):
                eb.set_text(new_legtxt[ix])
        ax.add_artist(leg1)
    
    
    if plot_legends[1]:
        legend_elements = [
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='Sto HP', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='s', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color="none",
                   label='Sto Multipurpose', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='^', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='RoR', markerfacecolor='none', markersize=10)]
        ax.legend(handles=legend_elements, **{
                'loc':'upper right', 
                'bbox_to_anchor': (1.2+offset, legend_y_coords[1]),
                'markerscale':1, 
                'title_fontsize':'medium', 
                'frameon':False,
                'fontsize':"small"})
        leg2 = ax.get_legend()
        leg2._legend_box.align = "left"
        leg2.set_title("HP Type")
        ax.add_artist(leg2)
    if plot_legends[2]:
        legend_elements = [
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='10MW', markerfacecolor='none', markersize=4),
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color="none",
                   label='100MW', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='1000MW', markerfacecolor='none', markersize=20)]
        ax.legend(handles=legend_elements, **{
                'loc':'upper right', 
                'bbox_to_anchor': (1.2+offset, legend_y_coords[2]),
                'markerscale':1.00, 
                'title_fontsize':'medium', 
                'frameon':False,
                'fontsize':'small'})
        leg3 = ax.get_legend()
        leg3._legend_box.align = "left"
        leg3.set_title("Mean Power Output")
    
    #You can use different 'cmaps' such as jet, plasm,magma, infereno,cividis, binary...(I simply chose cividis)
    ax.set_yticks([])
    ax.set_xticks([])
    x_spacing = 3
    y_spacing = 3
    #ax.set_xlim(data_gdf.total_bounds[0] - x_spacing/2, data_gdf.total_bounds[2] + x_spacing/2)
    #ax.set_ylim(data_gdf.total_bounds[1] - y_spacing/2, data_gdf.total_bounds[3] + y_spacing/2)
    if title is not None:
        ax.set_title(title, fontsize=title_font_size, loc='left', y=1.0, pad=-18)
    # Remove axes
    for pos in ('top', 'right', 'bottom', 'left'):
        ax.spines[pos].set_visible(False)
    # Label the reservoirs
    if res_label_field is not None:
        for x, y, label in zip(data['coordinates_1'], data['coordinates_0'], data[res_label_field]):
            ax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points', fontsize=5, alpha=0.7)

    return p1, p2, p3

In [ ]:
def calc_composite_metrics_radar_df(df: pd.DataFrame):
    sums = df[['Loss of forest, [%]',
       'Loss of agricultural land, [%]', 'GHG emissions, [%]',
       'HP Production, [%]']].sum()
    averages = df[['Firm Power Ratio, [%]']].mean()
    summary_series = pd.concat([sums, averages], axis=0)
    summary_series.index = [
        'Forest\nloss', 'Agri\nland\nloss', 'GHG\nEmissions', 'HP', 'Firm\nPower']
    return summary_series

In [ ]:
 def make_radar_plots(axs, df_radar: pd.DataFrame, optim_scenarios: pd.DataFrame) -> None:
    """ """
    optim_scenarios_keys = list(optim_scenarios.keys())
    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        if ix * 2 + 1 > len(optim_scenarios_keys):
            break
        key1 = optim_scenarios_keys[ix * 2]
        key2 = optim_scenarios_keys[ix * 2 + 1]
        dams1 = optim_scenarios[key1]
        dams2 = optim_scenarios[key2]

        marker_size = 10
        # Calculate the number of categories in the radar plot
        
        
        categories=list(df_radar)[:]
        N = len(categories)
        # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1] 
        
        scenarios = ("I", "II", "III")
        
        ax = axs[ix]

        handles = [
            Line2D([], [],  lw=0.4, c=color, marker="o", markersize=marker_size/6, label=species)
            for species, color in zip(["Built", "Not Built"], ['tab:blue', 'tab:orange'])
        ]

        if ix == 2:
            legend = ax.legend(
                handles=handles,
                prop={'size': 8},
                loc=(1, 0),       # bottom-right
                labelspacing=0.5, # add space between labels
                frameon=False     # don't put a frame
            )
            
        data1 = df_radar.loc[dams1]
        data1_summary = calc_composite_metrics_radar_df(data1)
        data2 = df_radar.loc[dams2]
        data2_summary = calc_composite_metrics_radar_df(data2)
        
        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)
        
        ax.set_yticks([])
        ax.set_xticks([])
        ax.yaxis.grid(True)
        ax.xaxis.grid(True)
        
        categories_abbr = list(data1_summary.index)
        ax.set_xticks(angles[:-1], categories_abbr, size=8)

        # First series
        values1=data1_summary.values.flatten().tolist()
        values1 += values1[:1]
        # Plot the first series
        ax.plot(angles, values1, linewidth=0.75, linestyle='solid', label="Built", alpha=0.5)
        ax.fill(angles, values1, 'tab:blue', alpha=0.1)
        ax.scatter(angles, values1, s=marker_size, zorder=10)
        # Second series
        values2=data2_summary.values.flatten().tolist()
        values2 += values2[:1]
        ax.plot(angles, values2, linewidth=0.75, linestyle='solid', label="Not Built", alpha=0.5)
        ax.fill(angles, values2, 'tab:orange', alpha=0.1)
        ax.scatter(angles, values2, s=marker_size, zorder=10)

        max_value = max(values1 + values2)
        #upper_limit = min([value for value in ytick_values if value > max_value])
        upper_limit = max_value
        ax.set_ylim(0, upper_limit)

        ax.set_xticks(angles[:-1])
        ax.tick_params(pad=3, size=7)
        
        ax.spines["start"].set_color("none")
        
        HANGLES = np.linspace(0, 2 * np.pi)
        
        H0 = np.zeros(len(HANGLES))
        H1 = np.ones(len(HANGLES)) * 20
        H2 = np.ones(len(HANGLES)) * 40
        H3 = np.ones(len(HANGLES)) * 60
        H4 = np.ones(len(HANGLES)) * 80
        H5 = np.ones(len(HANGLES)) * 100
        
        if upper_limit == 80:
            Hs = {0: H0, 40: H2, 80: H4}
        elif upper_limit == 100:
            Hs = {0: H0, 20: H1, 60: H3, 100:H5}
        else:
            Hs = {0: H0, 20: H1, 40: H2, 60: H3, 100:H5}

        for key, hangle in Hs.items():
            if key > upper_limit:
                break
            ax.plot(HANGLES, hangle, lw=0.8, ls=(0, (5, 6)), c='Grey')

        text_angle = -37.1
        PAD = 2
        
        ax.set_title(
            scenarios[ix], fontsize=14, loc='left', y=1.2, pad=-2, fontdict={'fontstyle': 'italic'})
        ax.tick_params(axis='x', labelsize=9)
        
        for level in Hs:
            if level == 0:
                continue
            if level > upper_limit:
                break
            text = f"{level}%"
            ax.text(text_angle, level + PAD, text, size=6, alpha=0.7)
    return

In [ ]:
def plot_rivord_histograms(axs, output_df: pd.DataFrame, optim_scenarios: pd.DataFrame):
    """ """
    optim_scenarios_keys = list(optim_scenarios.keys())

    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        if ix * 2 + 1 > len(optim_scenarios_keys):
            break
        key1 = optim_scenarios_keys[ix * 2]
        key2 = optim_scenarios_keys[ix * 2 + 1]
        dams1 = optim_scenarios[key1]
        dams2 = optim_scenarios[key2]

        data1 = output_df.loc[dams1]['RIV_ORD'].to_numpy()
        data2 = output_df.loc[dams2]['RIV_ORD'].to_numpy()

        combined_data = np.concatenate([data1, data2])
        # Determine the bins based on the combined data
        bins = np.histogram_bin_edges(combined_data, bins=8)
        bins = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5])
        logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))

        ax=axs[ix]
        hp_dist_colors = ["tab:blue", "tab:orange"]
        ax.hist([data1, data2], bins=bins, alpha = 0.5, edgecolor='black', linewidth=0.3,
                label=['Built', 'Not Built'], color=hp_dist_colors)
        
        if ix == 0:
            ax.set_ylim(0, 20)
        else:
            ax.set_ylim(0, 32)
        ax.set_xlim(2,8)
        ax.set_ylabel("Number of sites", fontsize=8)
        ax.set_xlabel("River Order", fontsize=8)
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(axis='y', labelsize=8)
        ax.set_xticks([2,3,4,5,6,7,8])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        if ix * 2 + 1 == len(optim_scenarios_keys) - 1:
            ax.legend(frameon=False, loc="upper left", bbox_to_anchor = (1.0, 1.1), 
                      prop={'size': 8})

In [ ]:
def plot_histograms(axs, output_df: pd.DataFrame, optim_scenarios: pd.DataFrame):
    """ """
    optim_scenarios_keys = list(optim_scenarios.keys())

    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        if ix * 2 + 1 > len(optim_scenarios_keys):
            break
        key1 = optim_scenarios_keys[ix * 2]
        key2 = optim_scenarios_keys[ix * 2 + 1]
        dams1 = optim_scenarios[key1]
        dams2 = optim_scenarios[key2]

        data1 = output_df.loc[dams1]['HP Production [GWh/year]'].to_numpy()
        data2 = output_df.loc[dams2]['HP Production [GWh/year]'].to_numpy()

        combined_data = np.concatenate([data1, data2])
        # Determine the bins based on the combined data
        bins = np.histogram_bin_edges(combined_data, bins=8)
        logbins=np.logspace(np.log10(1),np.log10(10000), 10)
        
        #logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))

        ax=axs[ix]
        hp_dist_colors = ["tab:blue", "tab:orange"]
        ax.hist([data1, data2], bins=logbins, alpha = 0.5, edgecolor='black', linewidth=0.3,
                label=['Built', 'Not Built'], color=hp_dist_colors)
        
        if ix == 0:
            ax.set_ylim(0, 15)
        else:
            ax.set_ylim(0, 20)
        ax.set_xlim(10, 10_000)
        #ax.set_xscale('log')
        ax.set_ylabel("Number of sites", fontsize=8)
        ax.set_xlabel("HP Production [GWh/year]", fontsize=8)
        ax.set_xticks([10,100,1_000,10_000])
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(axis='y', labelsize=8)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        if ix * 2 + 1 == len(optim_scenarios_keys) - 1:
            ax.legend(frameon=False, loc="upper left", bbox_to_anchor = (1.0, 1.1), 
                      prop={'size': 8})

In [ ]:
def plot_elev_dist(axs, output_df: pd.DataFrame, optim_scenarios: pd.DataFrame) -> None:
    """ """
    optim_scenarios_keys = list(optim_scenarios.keys())
    
    ix=2
    # Use the same bins for each plot
    key1 = optim_scenarios_keys[ix * 2]
    key2 = optim_scenarios_keys[ix * 2 + 1]
    dams1 = optim_scenarios[key1]
    dams2 = optim_scenarios[key2]

    data1 = df_combined.loc[dams1]['FSL (m)'].to_numpy()
    data2 = df_combined.loc[dams2]['FSL (m)'].to_numpy()

    combined_data = np.concatenate([data1, data2])
    # Determine the bins based on the combined data
    bins = np.histogram_bin_edges(combined_data, bins=10)    

    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        if ix * 2 + 1 > len(optim_scenarios_keys):
            break
        key1 = optim_scenarios_keys[ix * 2]
        key2 = optim_scenarios_keys[ix * 2 + 1]
        dams1 = optim_scenarios[key1]
        dams2 = optim_scenarios[key2]
        
        data1 = df_combined.loc[dams1]['FSL (m)'].to_numpy()
        data2 = df_combined.loc[dams2]['FSL (m)'].to_numpy()
        
        combined_data = np.concatenate([data1, data2])
        # Determine the bins based on the combined data
        #bins = np.histogram_bin_edges(combined_data, bins=10)

        ax=axs[ix]
        elev_dist_colors = ["tab:blue", "tab:orange"]
        ax.hist([data1, data2], bins=bins, alpha = 0.5, edgecolor='black', linewidth=0.3,
                label=['Built', 'Not Built'], color=elev_dist_colors)
        if ix == 0:
            ax.set_ylim(0, 15)
        else:
            ax.set_ylim(0, 25)
        ax.set_xlim(0, 1500)
        ax.set_ylabel("Number of sites", fontsize=8)
        ax.set_xlabel("Elevation, m.a.s.l.", fontsize=8)
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(axis='y', labelsize=8)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        if ix * 2 + 1 == len(optim_scenarios_keys) - 1:
            ax.legend(frameon=False, loc="upper left", bbox_to_anchor = (1, 1.1),
                     prop={'size': 8})   


## Create data for the radar plot

In [ ]:
import copy
df_radar = copy.deepcopy(
    output_df_riv_ord[[
        'forest_area_loss_km2', 'crop_area_loss_km2', 'Area, km2', 'GHG, tCO2eq/yr', 
        'Firm Power Ratio, [%]', 'HP Production [GWh/year]', 'RIV_ORD']])
# Metric 1
df_radar['Loss of forest, [%]'] = \
    df_radar['forest_area_loss_km2'] / (df_radar['forest_area_loss_km2']).sum() * 100
# Metric 2
df_radar['Loss of agricultural land, [%]'] = \
    df_radar['crop_area_loss_km2'] / (df_radar['crop_area_loss_km2']).sum() * 100
# Metric 3
df_radar['GHG emissions, [%]'] = \
    df_radar['GHG, tCO2eq/yr'] / df_radar['GHG, tCO2eq/yr'].sum() * 100
# Metric 4
# Firm Power Ratio, [%]
# Metric 5
# Percentage of total HP Production
df_radar['HP Production, [%]'] = \
    df_radar['HP Production [GWh/year]'] / df_radar['HP Production [GWh/year]'].sum() * 100
df_radar = df_radar.drop(columns = [
    'forest_area_loss_km2', 'crop_area_loss_km2', 'Area, km2', 'GHG, tCO2eq/yr', 'HP Production [GWh/year]',
])

### Plot the maps

In [ ]:
include_statistics = False
if include_statistics:
    plot_type = 'with_statistics'
else:
    plot_type: str = 'without_statistics'

In [ ]:
if plot_type == 'with_statistics':
    # warnings.filterwarnings('ignore')
    title_fontsize = 18
    fig, axs = plt.subplots(
        2,4, figsize = (9,8), 
        gridspec_kw=dict(
                left=.02, bottom=.02, right=.98, top=.98, wspace=.02, hspace=.0
            ),
        layout="constrained")
    scenario_id_to_latex = {
        "Ib" : "$I_b$",
        "Inb" : "$I_{nb}$",
        "IIb" : "$II_b$",
        "IInb" : "$II_{nb}$",
        "IIIb" : "$III_b$",
        "IIInb" : "$III_{nb}$"}
    column_name = 'GHG intensity cat [gCO2,eq/kWh]'
    vmin = 0
    vmax = 500

    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        data = output_df.loc[dams]
        if ix == 3:
            plt.subplots_adjust(right=0.9) 
            p1, p2, p3 = plot_mya_reservoirs(
                data = data, column_name = column_name, ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9, 
                plot_legends = (True, False, False),
                legend_y_coords = (1.0, 1.0, 1.0),
                offset=1.1)
        elif ix == 5:
            p1, p2, p3 = plot_mya_reservoirs(
                data = data, column_name = column_name, ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9, 
                plot_legends = (False, True, True),
                legend_y_coords = (1.0, 1.0, 0.7),
                offset=3.9)
        else:
            p1, p2, p3 = plot_mya_reservoirs(
            data = data, column_name = column_name, ax=axs.flat[ix], 
            title=scenario_id_to_latex[scenario_id],
            marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
            title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9,
            plot_legends=(False, False, False))

    # Remove the remaining (unused) axes
    for ax in axs.flat[-2:]:
        ax.remove()  
    gs = axs[1, 2].get_gridspec()
    statistics_grid = gs[1, 2:4]
    nrows = 3
    ncols = 3

    gs00 = statistics_grid.subgridspec(nrows, ncols, hspace=0.7,wspace=0.7)

    polar_axs = []
    cart_axs = []
    cart_axs2 = []
    for nrow in range(nrows):
        for ncol in range(ncols):
            if nrow == 0:
                polar_axs.append(fig.add_subplot(gs00[nrow, ncol], projection='polar'))
            elif nrow == 1:
                cart_axs.append(fig.add_subplot(gs00[nrow, ncol]))
            else:
                cart_axs2.append(fig.add_subplot(gs00[nrow, ncol]))

    make_radar_plots(polar_axs, df_radar, optim_scenarios)
    plot_histograms(cart_axs, output_df, optim_scenarios)
    plot_elev_dist(cart_axs2, output_df, optim_scenarios)

    # 'HP Production [GWh/year]'
    fig.tight_layout(pad=0.0)
    fig.show()
    fig.savefig(pathlib.Path('figures/moo/GHG_emissions_optim_maps_stats.png'), transparent=True, dpi=300)
    fig.savefig(pathlib.Path('figures/moo/GHG_emissions_optim_maps_stats.svg'))
    
else:
    title_fontsize = 18
    fig, axs = plt.subplots(
        2,3, figsize = (9,8), 
        gridspec_kw=dict(
                left=.02, bottom=.02, right=.98, top=.98, wspace=.02, hspace=.0
            ),
        layout="constrained")
    scenario_id_to_latex = {
        "Ib" : "$I_b$",
        "Inb" : "$I_{nb}$",
        "IIb" : "$II_b$",
        "IInb" : "$II_{nb}$",
        "IIIb" : "$III_b$",
        "IIInb" : "$III_{nb}$"}
    column_name = 'GHG intensity cat [gCO2,eq/kWh]'
    vmin = 0
    vmax = 500
    
    # reorder the optim_scenarios_dictionary
    desired_order = ['Ib', 'IIb', 'IIIb', 'Inb', 'IInb', 'IIInb']
    # Create a new dictionary with keys in the desired order
    reordered_scenarios = {key: optim_scenarios[key] for key in desired_order}

    for ix, (scenario_id, dams) in enumerate(reordered_scenarios.items()):
        """ """
        data = output_df.loc[dams]
        if ix == 3:
            plt.subplots_adjust(right=0.9) 
            p1, p2, p3 = plot_mya_reservoirs(
                data = data, column_name = column_name, ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9, 
                plot_legends = (True, False, False),
                legend_y_coords = (1.0, 1.0, 1.0),
                offset=1.1)
        elif ix == 5:
            p1, p2, p3 = plot_mya_reservoirs(
                data = data, column_name = column_name, ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9, 
                plot_legends = (True, True, True),
                legend_y_coords = (1.0, 1.0, 0.7),
                offset=1.1)
        else:
            p1, p2, p3 = plot_mya_reservoirs(
            data = data, column_name = column_name, ax=axs.flat[ix], 
            title=scenario_id_to_latex[scenario_id],
            marker_size = 'HP Production [GWh/year]', marker_size_multiplier=2.5, 
            title_font_size=title_fontsize, cmap='RdYlGn_r', vmin=0, vmax = 9,
            plot_legends=(False, False, False))

    # 'HP Production [GWh/year]'
    fig.tight_layout(pad=0.0)
    fig.show()
    fig.savefig(pathlib.Path('figures/moo/GHG_emissions_optim_maps_nostats.png'), transparent=True, dpi=600)
    fig.savefig(pathlib.Path('figures/moo/GHG_emissions_optim_maps_nostats.svg'))

In [ ]:
## Plot statistics on a separate plot
title_fontsize = 18
fig, axs = plt.subplots(
    3,3, figsize = (6.5,4), 
    gridspec_kw=dict(
            left=.02, bottom=.02, right=.98, top=.98, wspace=.02, hspace=.0
        ),
    layout="constrained")
for ax in axs.flat[:]:
    ax.remove()  
# Remove the remaining (unused) axes 
gs = axs[0,0].get_gridspec()
gs.update(wspace=1, hspace=1)
statistics_grid = gs[:, :]
nrows = 3
ncols = 3

gs00 = statistics_grid.subgridspec(nrows, ncols, hspace=0.45,wspace=0.6)

polar_axs = []
cart_axs = []
cart_axs2 = []
for nrow in range(nrows):
    for ncol in range(ncols):
        if nrow == 0:
            polar_axs.append(fig.add_subplot(gs00[nrow, ncol], projection='polar'))
        elif nrow == 1:
            cart_axs.append(fig.add_subplot(gs00[nrow, ncol]))
        else:
            cart_axs2.append(fig.add_subplot(gs00[nrow, ncol]))

make_radar_plots(polar_axs, df_radar.drop(columns=["RIV_ORD"]), optim_scenarios)
#plot_histograms(cart_axs, output_df_riv_ord, optim_scenarios)
plot_elev_dist(cart_axs, output_df_riv_ord, optim_scenarios)
plot_rivord_histograms(cart_axs2, output_df_riv_ord, optim_scenarios)


# 'HP Production [GWh/year]'
fig.tight_layout(pad=0.50)
fig.show()
fig.savefig(pathlib.Path('figures/moo/GHG_emissions_stats.png'), transparent=True, dpi=300)
fig.savefig(pathlib.Path('figures/moo/GHG_emissions_stats.svg'))

### Unused code